Error calcs for a single member for VWIND

In [12]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import warnings
from scipy import signal 
import glob as glob

from ecpaper_utils import bootstrap_utils as boot
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(boot)
importlib.reload(avg)

warnings.filterwarnings('ignore')

In [13]:
vwind_path="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

In [14]:
le_models = [x.split("_")[1].split(".")[0] for x in sorted(glob.glob(vwind_path+"/v300*.nc"))] ## parse files to get model names
le_dat = []

for le in le_models:
    print(le)
    ## vwind
    ds = xr.open_dataset(vwind_path+"v300_"+le+".nc")
    ## precip
    dsp = xr.open_dataset(vwind_path+"pr_"+le+".nc").rename({"Member": "member"})
    ## u100
    dsu = xr.open_dataset(vwind_path+"u100_"+le+".nc")
    ds = xr.merge([ds, dsp, dsu]).assign_coords({"Model": le})
    le_dat.append(ds)

CESM1-CAM5
CSIRO-Mk3-6-0
CanESM2
GFDL-CM3
MPI-ESM


In [15]:
le_dat = xr.concat(le_dat, dim = "Model")

Calculate the ensemble mean values for each LE

In [16]:
le_vdiff = []
le_vamp = []
le_pdiff_west = []
le_pdiff_south = []
le_pdiff_mex = []
le_udiff = []

for le in le_models: 
    print(le)
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all").mean(dim = "member")
    vpast_eddy = ds["vpast_djf"] - ds["vpast_djf"].mean("lon")
    vfut_eddy = ds["vfut_djf"] - ds["vfut_djf"].mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)   
    le_vdiff.append((vfutsw - vpastsw).values)
    
    # stationary wave ampltiude index
    vpast_k4to32 = filt.filterk(ds["vpast_djf"], 4, 32)
    vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
    le_vamp.append(vpast_amp.values)
        
    ## change in precipitation
    le_pdiff_west.append((ds["prf_west"] - ds["prp_west"]).values)
    le_pdiff_south.append((ds["prf_south"] - ds["prp_south"]).values)
    le_pdiff_mex.append((ds["prf_mex"] - ds["prp_mex"]).values)
    
    ## change in u100
    up = avg.cosweightlat(ds["upast_djf"],20,40)
    uf = avg.cosweightlat(ds["ufut_djf"],20, 40)
    le_udiff.append((uf - up).values)

CESM1-CAM5
CSIRO-Mk3-6-0
CanESM2
GFDL-CM3
MPI-ESM


Calculate errors for a single member.

In [17]:
nboots=1000
nmemp=1
nmemf=1

boots_vdiff = []
boots_udiff = []
boots_vamp = []
boots_pdiff_west = []
boots_pdiff_south = []
boots_pdiff_mex = []

for j, le in enumerate(le_models):
        
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all")
        
    ## bootstrapped V300
    vpast = boot.bootgen_multimem(ds["vpast_djf"], nmemp, seed=3, nboots = nboots)
    vfut = boot.bootgen_multimem(ds["vfut_djf"], nmemf, seed=3, nboots = nboots)
        
    vpast_eddy = vpast - vpast.mean("lon")
    vfut_eddy = vfut - vfut.mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)
        
    ## change in v300 SW
    boots_vdiff.append((vfutsw - vpastsw).values - le_vdiff[j])
        
    # stationary wave ampltiude index
    vpast_k4to32 = filt.filterk(vpast, 4, 32)
    vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
        
    boots_vamp.append(vpast_amp.values - le_vamp[j])
        
    ## u100 difference
    upast = boot.bootgen_multimem(ds["upast_djf"], nmemp, seed = 3, nboots = nboots)
    upast = avg.cosweightlat(upast,20,40)
    ufut = boot.bootgen_multimem(ds["ufut_djf"], nmemf, seed = 3, nboots = nboots)
    ufut = avg.cosweightlat(ufut,20, 40)
    boots_udiff.append(ufut.values - upast.values - le_udiff[j])
        
    ## calculate precip changes for three regions
    prp = boot.bootgen_multimem(ds["prp_west"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_west"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_west.append(prf.values - prp.values - le_pdiff_west[j])
        
    prp = boot.bootgen_multimem(ds["prp_south"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_south"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_south.append(prf.values - prp.values - le_pdiff_south[j])
        
    prp = boot.bootgen_multimem(ds["prp_mex"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_mex"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_mex.append(prf.values - prp.values - le_pdiff_mex[j])
    

boots_vdiff = np.concatenate(boots_vdiff)
boots_udiff = np.concatenate(boots_udiff)
boots_vamp = np.concatenate(boots_vamp)
boots_pdiff_west = np.concatenate(boots_pdiff_west)
boots_pdiff_south = np.concatenate(boots_pdiff_south)
boots_pdiff_mex = np.concatenate(boots_pdiff_mex)
    
sig_vdiff = np.std(boots_vdiff)
sig_udiff = np.std(boots_udiff)
sig_psi = np.std(boots_vamp)
sig_pdiff_west = np.std(boots_pdiff_west)
sig_pdiff_south = np.std(boots_pdiff_south)
sig_pdiff_mex = np.std(boots_pdiff_mex)
    
r_psi_vdiff = np.corrcoef(boots_vamp, boots_vdiff)[0,1]
r_udiff_vdiff = np.corrcoef(boots_udiff, boots_vdiff)[0,1]
r_psi_prwest = np.corrcoef(boots_vamp, boots_pdiff_west)[0,1]
r_psi_prsouth = np.corrcoef(boots_vamp, boots_pdiff_south)[0,1]
r_psi_prmex = np.corrcoef(boots_vamp, boots_pdiff_mex)[0,1]

In [73]:
# 5th to 95th percentile of internal variability range using the first 20 members from each
for j, le in enumerate(le_models):
        
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all")
    ds = ds.isel(member=slice(0,20))
    vpast_eddy = ds['vpast_djf'] - ds['vpast_djf'].mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    difs = np.zeros([500])
    count=0
    for imem in range(0,20,1):
        indices=np.argwhere(np.arange(0,20,1) !=imem)
        for imem2 in indices:
        #for imem2 in range(imem+1,20,1):
            difs[count] = vpastsw[imem2]-vpastsw[imem]
            count=count+1

    difs = difs[0:count]
    if (j == 0):
        difsall = np.zeros([5*count])
            
    difsall[j*count:(j+1)*count]=difs[:]
    
ivmin95=np.percentile(difsall,2.5)
ivmax95=np.percentile(difsall,97.5)
ivmin95 = xr.DataArray(ivmin95, name='ivmin95')
ivmax95 = xr.DataArray(ivmax95, name='ivmax95')

In [74]:
error_1mem = xr.Dataset({'sig_vdiff': xr.DataArray(sig_vdiff, name='sig_vdiff'), 
                        'sig_psi': xr.DataArray(sig_psi, name='sig_psi'), 
                        'sig_udiff': xr.DataArray(sig_udiff, name='sigg_udiff'),
                        'sig_pdiff_west': xr.DataArray(sig_pdiff_west, name='sigg_pdiff_west'), 
                        'sig_pdiff_south': xr.DataArray(sig_pdiff_south, name='sig_pdiff_south'), 
                        'sig_pdiff_mex': xr.DataArray(sig_pdiff_mex, name='sig_pdiff_mex'), 
                        'r_psi_vdiff': xr.DataArray(r_psi_vdiff, name='r_psi_vdiff'), 
                         'r_udiff_vdiff': xr.DataArray(r_udiff_vdiff, name='r_udiff_vdiff'),
                        'r_psi_prwest': xr.DataArray(r_psi_prwest, name='r_psi_prwest'), 
                        'r_psi_prsouth': xr.DataArray(r_psi_prsouth, name='r_psi_prsouth'), 
                        'r_psi_prmex': xr.DataArray(r_psi_prmex, name='r_psi_prmex'),
                        'ivmin95': xr.DataArray(ivmin95),
                        'ivmax95': xr.DataArray(ivmax95)})

error_1mem.to_netcdf("./DATA/errors_1mem.nc")